# Evaluation of the interval matrix exponential using Horner's method

Reference:

- https://www.mat.univie.ac.at/~neum/ms/intExp.pdf

In [1]:
using Revise, IntervalMatrices
using IntervalMatrices: _exp_remainder, _exp_remainder_series, scale_and_square

using LinearAlgebra
using LinearAlgebra: checksquare

In [2]:
# evaluate 
function H̃(K::Int, A::IntervalMatrix{T}) where {T}
    @assert K >= 1
    n = checksquare(A)
    Iₙ = IntervalMatrix(Interval(1.0) * I, n)
    out = Iₙ + A/K
    while K > 1
        K -= 1
        out = Iₙ + A/K * out
    end
    return out
end

# remainder, see eq. (4.4) in the paper
function R(K::Int, A::IntervalMatrix{T}) where {T}
    return _exp_remainder(A, 1.0, K)
    # NOTE: in the paper they use
    #_exp_remainder_series(A, 1.0, K)
    # but the former is more accurate
end

# cf. eq. (4.9) 
function H(K, A)
    H̃(K, A) + R(K,A)
end

H (generic function with 1 method)

In [3]:
function S(L, K, A)
    # scaling
    α = Interval(1/2)^L
    As = A * α
        
    # taylor evaluation with remainder using Horner's method
    T = H(K, As)

    # squaring
    out = copy(T)
    for i in 1:L
        out = square(out)
    end
    return out
end

S (generic function with 1 method)

In [ ]:
width(A::IntervalMatrix) = opnorm(diam.(A), Inf)

## Test case

In [22]:
function tline(;η=2, R₀=1.00, Rd₀=10.0, L₀=1e-10, C₀=1e-13 * 4.00, scale=1e-9)
    A₁₁ = zeros(η, η)
    A₁₂ = Bidiagonal(fill(-1/C₀, η), fill(1/C₀, η-1), :U)
    A₂₁ = Bidiagonal(fill(1/L₀, η), fill(-1/L₀, η-1), :L)
    A₂₂ = Diagonal(vcat(-Rd₀/L₀, fill(-R₀/L₀, η-1)))
    A  = [A₁₁ A₁₂; A₂₁ A₂₂] .* scale
    return A
end

R₀_int = Interval(0.99, 1.01)
Rd₀_int = Interval(9.9, 10.1)
L₀_int = Interval(1e-10, 1e-10)
C₀_int = Interval(3.99e-13, 4.01e-13)

[3.99e-13, 4.01001e-13]

In [27]:
# intervals of zero width
#A = IntervalMatrix(Matrix(tline(η=2)))

# intervals of non-zero width
A = IntervalMatrix(Matrix(tline(η=2, R₀=R₀_int, Rd₀=Rd₀_int,L₀=L₀_int,C₀=C₀_int)))

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [0, 0]                  [0, 0]                  …   [2493.76, 2506.27]   
 [0, 0]                  [0, 0]                     [-2506.27, -2493.76]  
     [9.99999, 10.0001]  [0, 0]                     [0, 0]                
   [-10.0001, -9.99999]      [9.99999, 10.0001]       [-10.1001, -9.89999]

In [28]:
S(10, 10, A)

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-6.11389e-06, 1.03058e-05]  …  [-0.000140555, 0.000170405]
 [-7.32309e-06, 4.97486e-06]     [-0.000113496, 0.000100246]
 [-3.24139e-07, 4.85592e-07]     [-6.7186e-06, 5.05224e-06] 
 [-6.67091e-07, 5.50309e-07]     [-5.32997e-06, 9.31041e-06]

In [29]:
exp_overapproximation(A, 1.0, 10)

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-1.57836e+121, 1.57836e+121]  …  [-2.19892e+122, 2.19892e+122]
 [-1.57836e+121, 1.57836e+121]     [-2.19892e+122, 2.19892e+122]
 [-1.57836e+121, 1.57836e+121]     [-2.19892e+122, 2.19892e+122]
 [-1.57836e+121, 1.57836e+121]     [-2.19892e+122, 2.19892e+122]

Note that the exp remainder of `A` bloats up; we *need* to do the scaling and squaring. 

In [31]:
_exp_remainder(A / 1000, 1.0, 10)

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-9.23394e-15, 9.23394e-15]  …  [-2.04953e-13, 2.04953e-13]
 [-9.23394e-15, 9.23394e-15]     [-2.04953e-13, 2.04953e-13]
 [-9.23394e-15, 9.23394e-15]     [-2.04953e-13, 2.04953e-13]
 [-9.23394e-15, 9.23394e-15]     [-2.04953e-13, 2.04953e-13]

In [32]:
_exp_remainder_series(A / 1000, 1.0, 10)

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-2.15941, 2.15941]  [-2.15941, 2.15941]  …  [-2.15941, 2.15941]
 [-2.15941, 2.15941]  [-2.15941, 2.15941]     [-2.15941, 2.15941]
 [-2.15941, 2.15941]  [-2.15941, 2.15941]     [-2.15941, 2.15941]
 [-2.15941, 2.15941]  [-2.15941, 2.15941]     [-2.15941, 2.15941]

In [35]:
S(10, 10, A)

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-6.11389e-06, 1.03058e-05]  …  [-0.000140555, 0.000170405]
 [-7.32309e-06, 4.97486e-06]     [-0.000113496, 0.000100246]
 [-3.24139e-07, 4.85592e-07]     [-6.7186e-06, 5.05224e-06] 
 [-6.67091e-07, 5.50309e-07]     [-5.32997e-06, 9.31041e-06]

In [36]:
exp_overapproximation(A, 1.0, 10) # bloats, it doesn't scale and square

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-1.57836e+121, 1.57836e+121]  …  [-2.19892e+122, 2.19892e+122]
 [-1.57836e+121, 1.57836e+121]     [-2.19892e+122, 2.19892e+122]
 [-1.57836e+121, 1.57836e+121]     [-2.19892e+122, 2.19892e+122]
 [-1.57836e+121, 1.57836e+121]     [-2.19892e+122, 2.19892e+122]

In [38]:
scale_and_square(A, 10, 1.0, 10)

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-6.07126e-06, 1.02484e-05]  …  [-0.000139791, 0.000169407]
 [-7.28151e-06, 4.93998e-06]     [-0.000112824, 9.9691e-05] 
 [-3.21743e-07, 4.82086e-07]     [-6.67145e-06, 5.01428e-06]
 [-6.62959e-07, 5.47146e-07]     [-5.29505e-06, 9.25511e-06]

## Case $\eta = 4$

In [39]:
# intervals of zero width
#A = IntervalMatrix(Matrix(tline(η=4)))

# intervals of non-zero width
A = IntervalMatrix(Matrix(tline(η=4, R₀=R₀_int, Rd₀=Rd₀_int,L₀=L₀_int,C₀=C₀_int)))

8×8 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [0, 0]                  [0, 0]                  …  [0, 0]                
 [0, 0]                  [0, 0]                     [0, 0]                
 [0, 0]                  [0, 0]                      [2493.76, 2506.27]   
 [0, 0]                  [0, 0]                     [-2506.27, -2493.76]  
     [9.99999, 10.0001]  [0, 0]                     [0, 0]                
   [-10.0001, -9.99999]      [9.99999, 10.0001]  …  [0, 0]                
 [0, 0]                    [-10.0001, -9.99999]     [0, 0]                
 [0, 0]                  [0, 0]                       [-10.1001, -9.89999]

In [44]:
H(10, A) + R(10, A) # bloats up without scale and square

8×8 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-2.02691e+133, 2.02691e+133]  …  [-2.24465e+134, 2.24465e+134]
 [-2.02691e+133, 2.02691e+133]     [-2.24465e+134, 2.24465e+134]
 [-2.02691e+133, 2.02691e+133]     [-2.24465e+134, 2.24465e+134]
 [-2.02691e+133, 2.02691e+133]     [-2.24465e+134, 2.24465e+134]
 [-2.02691e+133, 2.02691e+133]     [-2.24465e+134, 2.24465e+134]
 [-2.02691e+133, 2.02691e+133]  …  [-2.24465e+134, 2.24465e+134]
 [-2.02691e+133, 2.02691e+133]     [-2.24465e+134, 2.24465e+134]
 [-2.02691e+133, 2.02691e+133]     [-2.24465e+134, 2.24465e+134]

In [68]:
S(40, 40, A)

8×8 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-386.933, 447.615]   [-519.92, 488.658]    …  [-7282.67, 7696]    
 [-523.296, 491.804]   [-480.655, 599.987]      [-8867.59, 8466.61] 
 [-485.768, 503.823]   [-577.463, 541.683]      [-8158.17, 8552.05] 
 [-341.293, 327.916]   [-365.861, 390.689]      [-5740.28, 5523.73] 
  [-17.6605, 18.9071]   [-21.6403, 20.2895]      [-305.874, 298.202]
  [-32.152, 30.0271]    [-34.7302, 36.8044]  …   [-502.641, 519.921]
  [-31.6749, 33.3511]   [-38.1801, 36.847]       [-545.639, 509.294]
  [-29.9573, 28.3512]   [-32.7369, 34.2916]      [-407.663, 485.872]

In [51]:
width(ans)

73975.3662850389

In [70]:
scale_and_square(A, 20, 1.0, 20)

8×8 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-379.131, 438.589]   [-509.451, 478.786]   …  [-7134.66, 7541.77] 
 [-512.762, 481.871]   [-470.974, 587.936]      [-8690.32, 8294.93] 
 [-475.941, 493.65]    [-565.819, 530.712]      [-7992.26, 8380.42] 
 [-334.312, 321.207]   [-358.368, 382.715]      [-5623.52, 5409.93] 
  [-17.3013, 18.5273]   [-21.2065, 19.8767]      [-299.686, 292.179]
  [-31.5086, 29.4171]   [-34.0261, 36.0695]  …   [-492.487, 509.443]
  [-31.0318, 32.683]    [-37.417, 36.1001]       [-534.634, 498.965]
  [-29.3557, 27.7739]   [-32.0715, 33.6045]      [-399.407, 476.048]

In [53]:
width(ans)

81271.85570564584

width (generic function with 1 method)

In [ ]:
width()